# Extract data for Georgria boundary
We have previously listed, downloaded hailstorm data from NCDC site. Next we cleaned this data to weed out bad measurements. We extracted these CSV files and imported that into the GIS as feature classes. Now we will extract the data only for the state of Georgia by doing a boundary clipping operation.

In [1]:
import arcpy
from pathlib import Path
from glob import glob

List the input dataset paths

In [10]:
input_gdb = r'E:\GIS_Data\Analytics\Georgia_hailstones\decades_data_pro\decades_data_pro.gdb'
input_boundary = r'E:\GIS_Data\Analytics\Georgia_hailstones\decades_data_pro\\'+\
                'decades_hail_Georgia.gdb\Georgia_boundary'
output_fd = r'E:\GIS_Data\Analytics\Georgia_hailstones\decades_data_pro\\'+\
            'decades_hail_Georgia.gdb\hail_gcs'

## Clip to Georgia boundary

In [4]:
#first list all datasets in input geodatabase
arcpy.env.workspace = input_gdb
fc_list = arcpy.ListFeatureClasses()

In [11]:
for fc in fc_list:
    print("Extracting " + fc, end=" | ")
    output_fc = output_fd + "\\" + fc
    arcpy.analysis.Clip(fc, input_boundary, output_fc, None)
    print("done")

Extracting hail_1997 | done
Extracting hail_1998 | done
Extracting hail_1999 | done
Extracting hail_2000 | done
Extracting hail_2001 | done
Extracting hail_2002 | done
Extracting hail_2003 | done
Extracting hail_2004 | done
Extracting hail_2005 | done
Extracting hail_2006 | done
Extracting hail_2007 | done
Extracting hail_2008 | done
Extracting hail_2009 | done
Extracting hail_2010 | done
Extracting hail_2011 | done
Extracting hail_2012 | done
Extracting hail_2013 | done
Extracting hail_2014 | done
Extracting hail_2015 | done
Extracting hail_2016 | done


Let us list the contents of destination geodatabase to ensure data is created.

In [12]:
arcpy.env.workspace = output_fd
fc_list = arcpy.ListFeatureClasses()
for fc in fc_list:
    print(fc)

hail_1997
hail_1998
hail_1999
hail_2000
hail_2001
hail_2002
hail_2003
hail_2004
hail_2005
hail_2006
hail_2007
hail_2008
hail_2009
hail_2010
hail_2011
hail_2012
hail_2013
hail_2014
hail_2015
hail_2016


Now that data has been clipped to a smaller extend, further processing will be much faster and can be conducted just to this extent.

## Convert the time field
Let us list the fields in one our dataset and see if any of it has to be converted to a useful format.

In [13]:
fc1_fields = arcpy.ListFields(fc_list[0])
for field in fc1_fields:
    print(field.name + " | " + field.type)

OBJECTID | OID
Shape | Geometry
Field1 | Integer
F_ZTIME | Double
LON | Double
LAT | Double
WSR_ID | String
CELL_ID | String
RANGE | Integer
AZIMUTH | Integer
SEVPROB | Integer
PROB | Integer
MAXSIZE | Double
time_dates | Date


In [14]:
#loop through each feature class except first, 
# create a new time field and process the data from F_ZTIME field
fc_list = arcpy.ListFeatureClasses()
for fc in fc_list[1:]:
    print("Processing : " + fc, end=" | ")
    arcpy.management.ConvertTimeField(fc, "F_ZTIME", "yyyyMMdd", "time_dates", "DATE", "Not Used")
    print("done")

Processing : hail_1998 | done
Processing : hail_1999 | done
Processing : hail_2000 | done
Processing : hail_2001 | done
Processing : hail_2002 | done
Processing : hail_2003 | done
Processing : hail_2004 | done
Processing : hail_2005 | done
Processing : hail_2006 | done
Processing : hail_2007 | done
Processing : hail_2008 | done
Processing : hail_2009 | done
Processing : hail_2010 | done
Processing : hail_2011 | done
Processing : hail_2012 | done
Processing : hail_2013 | done
Processing : hail_2014 | done
Processing : hail_2015 | done
Processing : hail_2016 | done


## Project the data
The data downloaded from NCDC site is in geographic coordinate system. To process this in spatial analyst, we need to project this dataset so distances can be easily measured.

In [15]:
out_gdb = r'E:\GIS_Data\Analytics\Georgia_hailstones\decades_data_pro\\'+\
                'decades_hail_Georgia.gdb'

output_spatial_reference = "PROJCS['WGS_1984_Web_Mercator_Auxiliary_Sphere', "+\
                        "GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',"+\
                        "SPHEROID['WGS_1984',6378137.0,298.257223563]],"+\
                        "PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],"+\
                        "PROJECTION['Mercator_Auxiliary_Sphere'],PARAMETER['False_Easting',0.0],"+\
                        "PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',0.0],"+\
                        "PARAMETER['Standard_Parallel_1',0.0],PARAMETER['Auxiliary_Sphere_Type',0.0],"+\
                        "UNIT['Meter',1.0]]"
transformation = None
input_spatial_reference = "GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',"+\
                        "SPHEROID['WGS_1984',6378137.0,298.257223563]],"+\
                        "PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]]"

Let us create a timer so see how long each of these processing takes.

In [31]:
import time
def my_timer(start_time):
    now_time = time.time()
    elapsed = now_time - start_time
    
    hours, rem = divmod(elapsed, 3600)
    mins, sec = divmod(rem, 60)
    
    return "{0}:{1}:{2}".format(int(hours), int(mins), int(sec))

In [34]:
for fc in fc_list:
    start_time = time.time()
    print("Projecting " + fc, end=" | ")
    out_name = out_gdb + "\\hail_webmerc" + fc
    arcpy.management.Project(in_dataset = fc, out_dataset = out_name,
                            out_coor_system = output_spatial_reference, transform_method= None,
                            in_coor_system = input_spatial_reference)
    
    print("done : " + my_timer(start_time))

Projecting hail_1997 | done : 0:0:12
Projecting hail_1998 | done : 0:0:13
Projecting hail_1999 | done : 0:0:12
Projecting hail_2000 | done : 0:0:10
Projecting hail_2001 | done : 0:0:10
Projecting hail_2002 | done : 0:0:5
Projecting hail_2003 | done : 0:0:28
Projecting hail_2004 | done : 0:0:23
Projecting hail_2005 | done : 0:0:25
Projecting hail_2006 | done : 0:0:27
Projecting hail_2007 | done : 0:0:27
Projecting hail_2008 | done : 0:0:27
Projecting hail_2009 | done : 0:0:26
Projecting hail_2010 | done : 0:0:26
Projecting hail_2011 | done : 0:0:27
Projecting hail_2012 | done : 0:0:27
Projecting hail_2013 | done : 0:0:17
Projecting hail_2014 | done : 0:0:18
Projecting hail_2015 | done : 0:0:25
Projecting hail_2016 | done : 0:0:20
